<a href="https://colab.research.google.com/github/yutaka0321/private/blob/main/%E7%94%BB%E5%83%8F%E5%87%A6%E7%90%86%E3%82%A8%E3%83%B3%E3%82%B8%E3%83%B3(VAE%20and%20CNN).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## VAE

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Flatten, Reshape, Conv2D, Conv2DTranspose, BatchNormalization, LeakyReLU
from tensorflow.keras.models import Model
from tensorflow.keras.losses import binary_crossentropy
from tensorflow.keras import backend as K

# Define the VAE model
class VAE(tf.keras.Model):
    def __init__(self, latent_dim):
        super(VAE, self).__init__()
        self.latent_dim = latent_dim

        # Encoder with BatchNormalization and LeakyReLU
        self.encoder = tf.keras.Sequential([
            Input(shape=(64, 64, 3)),
            Conv2D(32, 3, strides=2, padding='same'),
            BatchNormalization(),
            LeakyReLU(alpha=0.2),
            Conv2D(64, 3, strides=2, padding='same'),
            BatchNormalization(),
            LeakyReLU(alpha=0.2),
            Flatten(),
            Dense(latent_dim + latent_dim)  # Mean and log variance
        ])

        # Decoder with BatchNormalization and LeakyReLU
        self.decoder = tf.keras.Sequential([
            Input(shape=(latent_dim,)),
            Dense(16 * 16 * 64, activation='relu'),
            Reshape((16, 16, 64)),
            Conv2DTranspose(64, 3, strides=2, padding='same'),
            BatchNormalization(),
            LeakyReLU(alpha=0.2),
            Conv2DTranspose(32, 3, strides=2, padding='same'),
            BatchNormalization(),
            LeakyReLU(alpha=0.2),
            Conv2DTranspose(3, 3, activation='sigmoid', padding='same')
        ])

    def call(self, inputs):
        mean, log_var = tf.split(self.encoder(inputs), num_or_size_splits=2, axis=1)
        z = self.reparameterize(mean, log_var)
        reconstruction = self.decoder(z)
        return reconstruction, mean, log_var

    def reparameterize(self, mean, log_var):
        epsilon = tf.random.normal(shape=mean.shape)
        return mean + tf.exp(0.5 * log_var) * epsilon

    def compute_loss(self, x, reconstruction, mean, log_var):
        reconstruction_loss = binary_crossentropy(K.flatten(x), K.flatten(reconstruction))
        kl_loss = -0.5 * tf.reduce_mean(log_var - tf.square(mean) - tf.exp(log_var) + 1)
        return tf.reduce_mean(reconstruction_loss + kl_loss)

# Prepare data with enhanced preprocessing
def preprocess_image(image):
    image = tf.image.resize(image, (64, 64))
    image = image / 255.0  # Normalize to [0, 1] range
    return image

def load_data():
    (x_train, _), (x_test, _) = tf.keras.datasets.cifar10.load_data()
    x_train = np.array([preprocess_image(img) for img in x_train])
    x_test = np.array([preprocess_image(img) for img in x_test])
    return x_train, x_test

# Train the VAE
def train_vae():
    x_train, x_test = load_data()
    latent_dim = 16  # Increased latent dimension for better representation
    vae = VAE(latent_dim)

    # Reduce the learning rate for better convergence
    vae.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0005))

    # Custom training loop
    @tf.function
    def train_step(x):
        with tf.GradientTape() as tape:
            reconstruction, mean, log_var = vae(x, training=True)
            loss = vae.compute_loss(x, reconstruction, mean, log_var)
        gradients = tape.gradient(loss, vae.trainable_variables)
        vae.optimizer.apply_gradients(zip(gradients, vae.trainable_variables))
        return loss

    for epoch in range(20):  # Increased epochs for more training
        epoch_loss = 0
        for i in range(0, len(x_train), 32):
            batch = x_train[i:i+32]
            loss = train_step(batch)
            epoch_loss += loss
        print(f"Epoch {epoch+1}, Loss: {epoch_loss.numpy()}")

    # Save the model weights after training
    vae.save_weights("vae_weights_updated.weights.h5")
    print("Model weights saved.")

    # Evaluation on test data
    test_loss = 0
    for i in range(0, len(x_test), 32):
        batch = x_test[i:i+32]
        reconstruction, mean, log_var = vae(batch, training=False)
        loss = vae.compute_loss(batch, reconstruction, mean, log_var)
        test_loss += loss
    print(f"Evaluation Loss: {test_loss.numpy() / len(x_test)}")

# Run training
train_vae()





/usr/local/lib/python3.10/dist-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


Epoch 1, Loss: 1077.8826904296875
Epoch 2, Loss: 1077.1873779296875
Epoch 3, Loss: 1077.270751953125
Epoch 4, Loss: 1077.2003173828125
Epoch 5, Loss: 1076.9918212890625
Epoch 6, Loss: 1077.1202392578125
Epoch 7, Loss: 1077.0576171875
Epoch 8, Loss: 1077.052001953125
Epoch 9, Loss: 1076.8382568359375
Epoch 10, Loss: 1077.0777587890625
Epoch 11, Loss: 1076.9215087890625
Epoch 12, Loss: 1076.9859619140625
Epoch 13, Loss: 1076.7645263671875
Epoch 14, Loss: 1076.787353515625
Epoch 15, Loss: 1077.0855712890625
Epoch 16, Loss: 1076.782470703125
Epoch 17, Loss: 1076.8358154296875
Epoch 18, Loss: 1076.855224609375
Epoch 19, Loss: 1076.8773193359375
Epoch 20, Loss: 1076.9703369140625
Model weights saved.
Evaluation Loss: 0.02157130432128906


## CNN

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical

# MNISTデータセットをロード
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# データをCNNに適合する形にリシェイプし、正規化
train_images = train_images.reshape((60000, 28, 28, 1)).astype('float32') / 255
test_images = test_images.reshape((10000, 28, 28, 1)).astype('float32') / 255

# ラベルをワンホットエンコーディング
train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

# CNNモデルを構築
model = models.Sequential()

# 畳み込み層
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))

# 全結合層
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

# モデルのコンパイル
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# モデルの訓練
model.fit(train_images, train_labels, epochs=5, batch_size=64)

# モデルの保存
model.save('mnist_cnn_model.h5')
print("モデルが 'mnist_cnn_model.h5' として保存されました")

# モデルの評価
test_loss, test_acc = model.evaluate(test_images, test_labels)
print(f"テスト精度: {test_acc}")

# モデルの読み込み（必要に応じて使用）
# loaded_model = tf.keras.models.load_model('mnist_cnn_model.h5')
# loaded_model.evaluate(test_images, test_labels)


11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 10s 5ms/step - accuracy: 0.8698 - loss: 0.4270
Epoch 2/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9835 - loss: 0.0528
Epoch 3/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - accuracy: 0.9893 - loss: 0.0366
Epoch 4/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.9909 - loss: 0.0288
Epoch 5/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.9926 - loss: 0.0215


モデルが 'mnist_cnn_model.h5' として保存されました
313/313 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.9883 - loss: 0.0366
テスト精度: 0.9908000230789185
